# Pregenerating coherence

If you need to generate many turbulence boxes with the same spatial geometry, you can pregenerate the spatial coherence and then save it to an HDF5 file for subsequent loading. This will make later simulations much faster. Here we will briefly show you how that is done.

## Preliminaries

Let's import packages and set up the simulation parameters we want. For details on the code here, see the example on constraining time series.

In [ ]:
%matplotlib inline
from pathlib import Path
import time

import matplotlib.pyplot as plt  # matplotlib for some plotting
import numpy as np  # numeric python functions
import pandas as pd  # need this to load our data from the csv files

from pyconturb import gen_turb, gen_spat_grid, TimeConstraint  # functions we need from PyConTurb
from pyconturb.coherence import generate_coherence_file
from pyconturb._utils import get_freq

coh_file = 'demo_coh.h5'

# simulation settings and variables
y = np.linspace(-65, 65, 15)  # lateral components of turbulent grid
z = np.linspace(14, 134, 15)  # vertical components of turbulent grid
spat_df = gen_spat_grid(y, z, comps=[0])  # create our spatial pandas dataframe. Columns are k, p_id x, y, and z.
kwargs = {'u_ref': 10, 'turb_class': 'B', 'z_hub': 70, 'l_c': 340.2,
          'T': 600, 'nt': 600, 'nf_chunk': 100, 'seed': 1337, 'dtype': np.float32}

def mytime(fun, *args, **kwargs):
    """Time a function!"""
    start = time.time()
    out = fun(*args, **kwargs)
    duration = time.time() - start
    return out, duration    

## Turbulence without a pre-generated coherence file

In [ ]:
turb_df_nofile, duration_nofile = mytime(gen_turb, spat_df, **kwargs)

print(f'Simulation time: {duration_nofile:.1f}s')

## Generate coherence file

In [ ]:
t, freq = get_freq(**kwargs)
generate_coherence_file(spat_df, coh_file, freq=freq, verbose=True, **kwargs)

## Generate turbulence using coherence file

In [ ]:
turb_df_file, duration_file = mytime(gen_turb, spat_df, **{**kwargs, **{'coh_file': coh_file}})

print(f'Simulation time: {duration_file:.1f}s')
print(f'\nWith pre-generated coherence file, simulation is: {duration_nofile/duration_file:.2f} times faster!')

## Verify turbulence is the same

If this statement below does not throw an error, then the two turbulence fields are identical.

In [ ]:
pd.testing.assert_frame_equal(turb_df_nofile, turb_df_file)